### Find intersection

## Equations:
#### - reservoir line :
- P - Gs * D = (Pwf - Gs * D_total)
#### - casing line:
- P - (Pcs/40,000) * D = Pcs

    

In [4]:
import numpy as np
import pandas as pd 

In [23]:
def get_equations(Pcs,j,BHSP,Q,Gs,D_total):
    """
    Pcs : surface injection pressure (psi) 
    j : productivity index 
    BHSP : bottomhole static pressure (psi)
    Q : oil flow rate ( bbl / day)
    Gs : static gradient ( psi/ft )
    D_total : total depth to the perforation (Ft)
    return 
       1- equation of reservoir line
       2- equation of caisng line
    """
    # so that we get injection point
    # the gradient of the surface injection pressure increases so take the gradint of the Pcs put reduce the 
    # intercept 
    # the equation is : ( P = Pcs + ( Pcs/40,000) * D )
    # from the equation the gradient changes with the pressure so , take PCs/40,000 
    # so the equation to get the ingection pint will be  P =(Pcs-100) + (Pcs/40,000) * D
#     Pcs= Pcs-100
    Gcs = Pcs/40000
    Pwf= BHSP-(Q/j)
    # reservoir line
    reservoir_line = [1 ,-Gs, (Pwf - (Gs* D_total))]
    # casing line
    casing_line = [1,-Gcs,Pcs-100]
    
    return reservoir_line, casing_line

In [24]:
# get_equations()

In [26]:
reservoir_line,casing_line= get_equations(870,2.55,2000,600,.5,7500)
reservoir_line,casing_line

([1, -0.5, -1985.2941176470588], [1, -0.02175, 770])

In [8]:
def find_intersection(line_1,line_2):
    """
    line_1 : coef of line_1 , in the form (a*P + b*Depth = c) , put list of [a1,b1,c1]
    line_2 : coef of line_2 , in the form (a*P + b*Depth = c) , put list of [a2,b2,c2]
    return : X,Y of the intersection
    """
    Z= np.array([ [line_1[0],line_1[1]],[line_2[0],line_2[1]] ])
    X= np.array([ [line_1[-1],line_1[-2]],[line_2[-1],line_2[-2]] ])
    Y= np.array([ [line_1[0],line_1[-1]],[line_2[0],line_2[-1]] ])
    try:
        D = np.linalg.det(Z)
        Dx= np.linalg.det(X)
        Dy= np.linalg.det(Y)
        P= Dx/D
        Depth= Dy/D 
        return np.round(Depth,2) , np.round(P,2)
    except:
        print("Error, there is no intersection")

In [9]:
find_intersection([1,-.4,-1200],[1,-0.03,1200])

(6486.49, 1394.59)

In [27]:
find_intersection(reservoir_line,casing_line)

(5761.2, 895.31)

In [28]:
def get_GF1_GF2(Tubing_head,Pcs,injection_depth, injection_pressure):
    """
    Tubing_head : tubing head pressure (Psi)
    Pcs : surface casing pressure (Psi)
    injection_depth : depth at the pint of injection (Ft)
    injection_pressure : pressure of the reservoir at the injection point (Psi)
    return : 
        Gf1 -> of the line from the wellhead to injection point
        Gf2 -> of the line from the modified wellhead to injection point
        Pwh2 -> modified wellhead pressure
    """
    Gf1 = (injection_pressure - Tubing_head) / injection_depth
    Pwh2= Tubing_head + 0.2*(Pcs - Tubing_head)
    Gf2=  (injection_pressure - Pwh2) / injection_depth
    return round(Gf1,3) , round(Gf2,3), round(Pwh2,3)

In [29]:
# get_GF1_GF2()
get_GF1_GF2(100,870,5731.241014346458,880.3263895261689)

(0.136, 0.109, 254.0)

## Equations:
#### - gas line :
- P - (Pcs/40,000) = Pcs
#### - first line of the valves:
- P - (Glf * D) = Pwh1
#### -following lines lines:
- P - (Glf * D) = (Pwh2+ Gf2*Di-1) 
    

In [69]:
# the main loop
def get_spacings(Pwh1,Pwh2,Pcs,Pko,Glf,Gf2,injection_depth,injection_pressure):
    depths=[]
    P1=[]
    P2=[]  
    casing_ko=[1,(-Pko/40000),Pko]
    line_1= [1,-Glf,Pwh1]
    depth,p1= find_intersection(casing_ko,line_1)
    depths.append(depth)
    P1.append(p1)
    p2= Pwh2 + Gf2*depths[-1]
    P2.append(p2)
    # new_line
    Gcs = Pcs/40000
#     for i in range(30):
    while depths[-1] < injection_depth:
        Pc_0 = Pcs*(1+(depths[-1]/40000))
#         casing_ps=[1,-Pc_0/40000,Pc_0]
        casing_ps=[1,-Gcs,Pc_0]
        line_2= [1,-Glf,P2[-1]]
        depth,p1= find_intersection(casing_ps,line_2)
        depths.append(depth + depths[-1])
        P1.append(p1)
        p2= Pwh2 + Gf2*depths[-1]
        P2.append(p2)
    dics={
    "Depth":np.round(depths,2),
     "P1":np.round(P1,2),
     "P2":np.round(P2,2)
     }

    df =pd.DataFrame(dics)
    df.drop(axis=0,index=df.index[-1],inplace=True)
    final_depth=np.round(injection_depth,2)
    p1= Pcs * (1+ (final_depth / 40000))
    p2= injection_pressure
    final={
    "Depth":np.round(injection_depth,2),
     "P1":np.round(p1,2),
     "P2":np.round(p2,2)
     }
    
    df=df.append(final,ignore_index=True)
    # new lines 
    if (df.loc[df.index[-1],"Depth"] - df.loc[df.index[-2],"Depth"])  < 200:
        df.drop(axis=0,index=df.index[-2],inplace=True)
        
    # write the number of valves
    valves= [i+1 for i in range(len(df["Depth"]))]
    df["Valve .No"]= valves
    # df=df[["Valve .No","Depth","P1","P2"]]
    cols=["Valve .No","Depth","P1","P2"]
    df=df.reindex(columns=cols)
    return df 


In [72]:
df=get_spacings(100,254.0,870,920,.5, 0.10928285653287778,5761.2,895.31)

In [73]:
df 

,Valve .No,Depth,P1,P2
0,1,1719.08,959.54,441.87
1,2,2692.47,928.56,548.24
2,3,3487.70,945.86,635.15
3,4,4137.38,959.99,706.14
4,5,4668.16,971.53,764.15
5,6,5101.79,980.96,811.54
6,7,5456.05,988.67,850.25
8,8,5761.20,995.31,895.31


In [62]:
depths 

[1719.08,
 2694.38,
 3491.71,
 4143.25,
 4675.46,
 5110.06,
 5464.870000000001,
 5754.4800000000005,
 5990.830000000001]

In [40]:
df.loc[df.index[-1],"Depth"] - df.loc[df.index[-2],"Depth"]

15.729999999999563

In [43]:
casing=[1,(-870/40000),870]
line_2= [1,-.5,440]
depth,p1= find_intersection(casing,line_2) 
p2= 254 + .10928285653287778*2600

In [44]:
depth

899.1113434396234

In [13]:
p1

889.5556717198117

In [14]:
p2 

538.1354269854822

In [ ]:
def get_temperature_gradient(Tre,Ts,total_depth):
    g= (Tre-Ts) / total_depth
    

In [141]:
def all_in_one(total_depth,wellhead_pressure, Pcs ,Pko,Glf,Gs,Q,BHSP,J,Tre,Ts,R):
    """
    total_depth : total depth to the pay zone  ( ft )
    wellhead_pressure : tuning pressure at the surface ( psi )
    Pcs : surface casing pressure ( injection pressure) ( psi )
    Pko : kick_off pressure if it exists ( psi )
    Glf : load flowing gradient ( psi/ft )
    Gs : static gradient ( psi/ft )
    Q : oil flow rate ( bbl/day)
    BHSP : bottom_hole static pressure -> reservoir pressure
    J : productivity index 
    Tres : reservoir temperature ( F )
    Ts : surface following temperature ( F )
    R: port size 
    """
    reservoir_line,casing_line = get_equations(Pcs,J,BHSP,Q,Gs,total_depth)
    injection_depth,injection_pressure = find_intersection(reservoir_line,casing_line)
    Gf1,Gf2,Pwh2 = get_GF1_GF2(wellhead_pressure,Pcs,injection_depth,injection_pressure)
    df=get_spacings(wellhead_pressure,Pwh2,Pcs,Pko,Glf,Gf2,injection_depth,injection_pressure) 
    Tg= ((Tre-Ts) / total_depth)
    df["Temp"] = np.round(Ts + Tg * df["Depth"],2)
    df["Pdt"] = np.round((1-R)*df["P1"] + R*df["P2"],2)
    return df

In [142]:
df_2=all_in_one(7500,100,870,920,.5,.5,600,2000,2.55,180,100,.1534)

In [143]:
df_2

,Depth,P1,P2,Temp,Pdt
0,1719.08,959.54,441.38,118.34,880.05
1,2695.40,929.54,547.80,128.75,870.98
2,3494.14,947.17,634.86,137.27,899.26
3,4147.31,961.44,706.06,144.24,922.26
4,4681.24,973.02,764.26,149.93,941.00
5,5117.57,982.42,811.82,154.59,956.25
6,5474.04,990.05,850.67,158.39,968.67
7,5731.24,995.39,880.33,161.13,977.74
